In [1]:
from __future__ import print_function

import os.path
import pandas as pd

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']

# The ID and range of a sample spreadsheet.
SAMPLE_SPREADSHEET_ID = '14y0FLqGTA6rgvfAmwBeH8bUf_wBKDEHTH8JxXwxtSJI'
SAMPLE_RANGE_NAME = 'FRADEE!A1:Z10000'



In [2]:
def chama_frade():
    """Shows basic usage of the Sheets API.
    Prints values from a sample spreadsheet.
    """
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    try:
        service = build('sheets', 'v4', credentials=creds)

        # Call the Sheets API
        sheet = service.spreadsheets()
        result = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                                    range=SAMPLE_RANGE_NAME).execute()
        values = result.get('values', [])
        values = result.get('values', [])

        
    except HttpError as err:
        print(err)
    
    dados_frade = pd.DataFrame(data=values, columns=values.pop(0))

    return dados_frade

In [3]:
df = chama_frade()
df.columns

Index(['', '', '', 'Ticket', 'NF', ' Reposição', 'Data Ocorrência', '',
       'Ocorrência', 'Ressalva/Recusa', 'Cliente avisou no prazo',
       'Contato  com a transp. em 24h?', '', 'Parceiro', 'Embalagem',
       ' Valor ', ' Produto ', '', 'Emissão', 'PAF', 'Status',
       'Em Aberto ou Finalizado?', 'Observação', 'Quantidade', 'Origem',
       'Categoria Produto'],
      dtype='object')

In [4]:
df.head(2)

,,,,Ticket,NF,Reposição,Data Ocorrência,,Ocorrência,Ressalva/Recusa,Cliente avisou no prazo,Contato com a transp. em 24h?,,Parceiro,Embalagem,Valor,Produto,,Emissão,PAF,Status,Em Aberto ou Finalizado?,Observação,Quantidade,Origem,Categoria Produto
0,,,,#9103,36985,-,10/06/2020,,Retorno,Sim,Sim,Sim,,Alfa,Caixa Papelão,"R$ 239,78",Painel Solar Fotovoltaico Resun RS6E-155M (155Wp),,21/08/2024,Produto,Isenta/Recusa,Finalizado,"Acordo de Isenção, fora do prazo, sem recusa e...",1,MG,Painel 155 Wp
1,,,,#10358,46732,47358,14/08/2020,,Avaria,Sim,Sim,Sim,,Viktoria,Caixa Papelão,"R$ 863,18",PAINEL SOLAR FOTOVOLTAICO ZTROON ZTP340P (340WP),,16/08/2024,Produto,Isenta/Recusa,Finalizado,Acordo de Isenção,2,MG,Painel 340 Wp


In [5]:
df2 = df[['Ticket','NF','Data Ocorrência','Ocorrência','Parceiro',' Valor ',' Produto ','Emissão','Quantidade','Categoria Produto']]
df2.head(5)

,Ticket,NF,Data Ocorrência,Ocorrência,Parceiro,Valor,Produto,Emissão,Quantidade,Categoria Produto
0,#9103,36985,10/06/2020,Retorno,Alfa,"R$ 239,78",Painel Solar Fotovoltaico Resun RS6E-155M (155Wp),21/08/2024,1,Painel 155 Wp
1,#10358,46732,14/08/2020,Avaria,Viktoria,"R$ 863,18",PAINEL SOLAR FOTOVOLTAICO ZTROON ZTP340P (340WP),16/08/2024,2,Painel 340 Wp
2,#10567,46841,25/08/2020,Avaria,Gran Cargo,"R$ 509,52",PAINEL SOLAR FOTOVOLTAICO RENESOLA (555WP) BR,15/08/2024,1,Painel 555 Wp
3,#10755,45974,01/09/2020,Extravio,Tecmar,"R$ 863,18",Painel Solar Fotovoltaico BYD 340-PHK-36 Poli ...,09/08/2024,2,Painel 340 Wp
4,#36184,9196,08/01/2024,Extravio,Tecmar,"R$ 8.276,07",BATERIA DE Lítio UNIPOWER UPLFP48-100 (100AH) ...,01/08/2024,1,Bateria 100 Ah


In [6]:
fradee = df2[df2[' Produto '] != 'Carga Total']
fradee.head(5)

,Ticket,NF,Data Ocorrência,Ocorrência,Parceiro,Valor,Produto,Emissão,Quantidade,Categoria Produto
0,#9103,36985,10/06/2020,Retorno,Alfa,"R$ 239,78",Painel Solar Fotovoltaico Resun RS6E-155M (155Wp),21/08/2024,1,Painel 155 Wp
1,#10358,46732,14/08/2020,Avaria,Viktoria,"R$ 863,18",PAINEL SOLAR FOTOVOLTAICO ZTROON ZTP340P (340WP),16/08/2024,2,Painel 340 Wp
2,#10567,46841,25/08/2020,Avaria,Gran Cargo,"R$ 509,52",PAINEL SOLAR FOTOVOLTAICO RENESOLA (555WP) BR,15/08/2024,1,Painel 555 Wp
3,#10755,45974,01/09/2020,Extravio,Tecmar,"R$ 863,18",Painel Solar Fotovoltaico BYD 340-PHK-36 Poli ...,09/08/2024,2,Painel 340 Wp
4,#36184,9196,08/01/2024,Extravio,Tecmar,"R$ 8.276,07",BATERIA DE Lítio UNIPOWER UPLFP48-100 (100AH) ...,01/08/2024,1,Bateria 100 Ah


In [7]:
fradee['Emissão'] = pd.to_datetime(fradee['Emissão'], format='%d/%m/%Y', errors='coerce')
fradee = fradee.sort_values(by='Emissão', ascending=False)
fradee['Emissão'] = fradee['Emissão'].dt.strftime('%d/%m/%Y')

C:\Users\Daniel\AppData\Local\Temp\ipykernel_6452\2092292157.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fradee['Emissão'] = pd.to_datetime(fradee['Emissão'], format='%d/%m/%Y', errors='coerce')


In [8]:
fradee.head(5)

,Ticket,NF,Data Ocorrência,Ocorrência,Parceiro,Valor,Produto,Emissão,Quantidade,Categoria Produto
1304,#41080,24753,10/09/2024,Avaria,Viktoria,"R$ 482,67",Painel Solar Fotovoltaico Luxen Lnvu-555M (555Wp),02/09/2024,1,Painel 555 Wp
1303,#41024,24718,06/09/2024,Avaria,Solistica,"R$ 602,00",PAINEL SOLAR FOTOVOLTAICO LUXEN LNCT-585N TOPC...,30/08/2024,1,Painel 585 Wp
0,#9103,36985,10/06/2020,Retorno,Alfa,"R$ 239,78",Painel Solar Fotovoltaico Resun RS6E-155M (155Wp),21/08/2024,1,Painel 155 Wp
1,#10358,46732,14/08/2020,Avaria,Viktoria,"R$ 863,18",PAINEL SOLAR FOTOVOLTAICO ZTROON ZTP340P (340WP),16/08/2024,2,Painel 340 Wp
2,#10567,46841,25/08/2020,Avaria,Gran Cargo,"R$ 509,52",PAINEL SOLAR FOTOVOLTAICO RENESOLA (555WP) BR,15/08/2024,1,Painel 555 Wp


In [9]:
fradee.shape

(1241, 10)

In [10]:
mestre_bu = pd.read_parquet(r"N:\Drives compartilhados\28 - Dados BI\2 Dados Transformados\2. Vendas\2.NFs_Venda.parquet")
mestre_bu.columns

Index(['Núm. NF-e', 'Data de Emissão', 'Natureza de Operação', 'Núm. Pedido',
       'Data da Venda', 'Nome do Contato', 'Tipo Pessoa', 'CPF/CNPJ',
       'ID Cliente', 'Identificador Contribuinte',
       ...
       'BU Report', 'BU Por Vendedor Report Diário',
       'BU Produto Report Diário', 'CUSTOGER', 'Rateio Cartao/Boleto',
       'Forma de PGTO Simplificada', 'Filtrar Gerador', 'GERENTE',
       'Saldo e Custo Estoque.Custo Médio', 'chave_recorrentes'],
      dtype='object', length=123)

In [11]:
mestre_bu = mestre_bu[['Núm. NF-e', 'Data de Emissão','UF Destino']]
mestre_bu.head(3)

,Núm. NF-e,Data de Emissão,UF Destino
0,25587,2024-09-17,PA
1,25582,2024-09-17,RJ
2,25642,2024-09-17,MG


In [12]:
mestre_bu['Data de Emissão'] = pd.to_datetime(mestre_bu['Data de Emissão'], format='%d/%m/%Y', errors='coerce')
mestre_bu = mestre_bu.sort_values(by='Data de Emissão', ascending=False)
mestre_bu['Data de Emissão'] = mestre_bu['Data de Emissão'].dt.strftime('%d/%m/%Y')

mestre_bu.rename(columns={'Núm. NF-e': 'NF', 'Data de Emissão': 'Emissão'}, inplace=True)
mestre_bu.head(3)

,NF,Emissão,UF Destino
0,25587,17/09/2024,PA
115,25598,17/09/2024,PI
107,25574,17/09/2024,PR


In [13]:
primeiros_bu = mestre_bu.groupby('NF').first().reset_index()
df_resultado = pd.merge(fradee,primeiros_bu, on=['NF'], how='left')
df_resultado.head(10)

,Ticket,NF,Data Ocorrência,Ocorrência,Parceiro,Valor,Produto,Emissão_x,Quantidade,Categoria Produto,Emissão_y,UF Destino
0,#41080,24753,10/09/2024,Avaria,Viktoria,"R$ 482,67",Painel Solar Fotovoltaico Luxen Lnvu-555M (555Wp),02/09/2024,1,Painel 555 Wp,02/09/2024,SP
1,#41024,24718,06/09/2024,Avaria,Solistica,"R$ 602,00",PAINEL SOLAR FOTOVOLTAICO LUXEN LNCT-585N TOPC...,30/08/2024,1,Painel 585 Wp,30/08/2024,SP
2,#9103,36985,10/06/2020,Retorno,Alfa,"R$ 239,78",Painel Solar Fotovoltaico Resun RS6E-155M (155Wp),21/08/2024,1,Painel 155 Wp,NaN,<NA>
3,#10358,46732,14/08/2020,Avaria,Viktoria,"R$ 863,18",PAINEL SOLAR FOTOVOLTAICO ZTROON ZTP340P (340WP),16/08/2024,2,Painel 340 Wp,10/08/2020,SP
4,#10567,46841,25/08/2020,Avaria,Gran Cargo,"R$ 509,52",PAINEL SOLAR FOTOVOLTAICO RENESOLA (555WP) BR,15/08/2024,1,Painel 555 Wp,12/08/2020,RS
5,#10755,45974,01/09/2020,Extravio,Tecmar,"R$ 863,18",Painel Solar Fotovoltaico BYD 340-PHK-36 Poli ...,09/08/2024,2,Painel 340 Wp,23/07/2020,PA
6,#36184,9196,08/01/2024,Extravio,Tecmar,"R$ 8.276,07",BATERIA DE Lítio UNIPOWER UPLFP48-100 (100AH) ...,01/08/2024,1,Bateria 100 Ah,07/12/2023,MG
7,#11407,49871,25/09/2020,Extravio,Alfa,"R$ 287,48",Painel Solar Fotovoltaico Resun RSM020P (20Wp),29/07/2024,4,Painel 20 Wp,03/09/2020,TO
8,#11533,50133,02/10/2020,Extravio,Alfa,"R$ 7.224,00",PAINEL SOLAR FOTOVOLTAICO LUXEN LNCT-585N TOPC...,24/07/2024,12,Painel 585 Wp,10/09/2020,PI
9,#11565,50233,18/10/2020,Retorno,Alfa,"R$ 868,22",PAINEL SOLAR FOTOVOLTAICO SUNOVA SS-460-60MDH ...,17/07/2024,2,Painel 460 Wp,14/09/2020,MG


In [14]:
produtos = pd.read_parquet(r"N:\Drives compartilhados\28 - Dados BI\2 Dados Transformados\2. Produtos\D_Produtos NeoSolar.parquet")
produtos = produtos[['Nome Produto', 'Codigo SKU']]
produtos.head(3)

,Nome Produto,Codigo SKU
0,Configurador Rápido Epever FC-01,10128
1,CABO FOTOVOLTÁICO LAFEBER 6MM² VERMELHO,20021
2,Estrutura Painel Romagnole Fibrocimento 411277...,10000871


In [15]:
produtos.rename(columns={'Nome Produto': ' Produto ', 'Codigo SKU': 'SKU'}, inplace=True)
produtos.head(5)

,Produto,SKU
0,Configurador Rápido Epever FC-01,10128
1,CABO FOTOVOLTÁICO LAFEBER 6MM² VERMELHO,20021
2,Estrutura Painel Romagnole Fibrocimento 411277...,10000871
3,Controlador de Carga com display Epever ViewSt...,10000446
4,Estrutura Romagnole 411111 Rs-225 4 Paineis Fi...,10001120


In [16]:
df_resultado.head(20)

,Ticket,NF,Data Ocorrência,Ocorrência,Parceiro,Valor,Produto,Emissão_x,Quantidade,Categoria Produto,Emissão_y,UF Destino
0,#41080,24753,10/09/2024,Avaria,Viktoria,"R$ 482,67",Painel Solar Fotovoltaico Luxen Lnvu-555M (555Wp),02/09/2024,1,Painel 555 Wp,02/09/2024,SP
1,#41024,24718,06/09/2024,Avaria,Solistica,"R$ 602,00",PAINEL SOLAR FOTOVOLTAICO LUXEN LNCT-585N TOPC...,30/08/2024,1,Painel 585 Wp,30/08/2024,SP
2,#9103,36985,10/06/2020,Retorno,Alfa,"R$ 239,78",Painel Solar Fotovoltaico Resun RS6E-155M (155Wp),21/08/2024,1,Painel 155 Wp,NaN,<NA>
3,#10358,46732,14/08/2020,Avaria,Viktoria,"R$ 863,18",PAINEL SOLAR FOTOVOLTAICO ZTROON ZTP340P (340WP),16/08/2024,2,Painel 340 Wp,10/08/2020,SP
4,#10567,46841,25/08/2020,Avaria,Gran Cargo,"R$ 509,52",PAINEL SOLAR FOTOVOLTAICO RENESOLA (555WP) BR,15/08/2024,1,Painel 555 Wp,12/08/2020,RS
5,#10755,45974,01/09/2020,Extravio,Tecmar,"R$ 863,18",Painel Solar Fotovoltaico BYD 340-PHK-36 Poli ...,09/08/2024,2,Painel 340 Wp,23/07/2020,PA
6,#36184,9196,08/01/2024,Extravio,Tecmar,"R$ 8.276,07",BATERIA DE Lítio UNIPOWER UPLFP48-100 (100AH) ...,01/08/2024,1,Bateria 100 Ah,07/12/2023,MG
7,#11407,49871,25/09/2020,Extravio,Alfa,"R$ 287,48",Painel Solar Fotovoltaico Resun RSM020P (20Wp),29/07/2024,4,Painel 20 Wp,03/09/2020,TO
8,#11533,50133,02/10/2020,Extravio,Alfa,"R$ 7.224,00",PAINEL SOLAR FOTOVOLTAICO LUXEN LNCT-585N TOPC...,24/07/2024,12,Painel 585 Wp,10/09/2020,PI
9,#11565,50233,18/10/2020,Retorno,Alfa,"R$ 868,22",PAINEL SOLAR FOTOVOLTAICO SUNOVA SS-460-60MDH ...,17/07/2024,2,Painel 460 Wp,14/09/2020,MG


In [17]:
# Encontre o primeiro elemento de cada grupo na tabela "produtos"
primeiros_produtos = produtos.groupby(' Produto ').first().reset_index()
primeiros_produtos.head(5)

,Produto,SKU
0,Bateria Estacionária Fulguris FGCL 220 D (220Ah),10001277
1,*KIT TESTE PORTAL B2B NÃO UTILIZAR*,21643
2,"10 LITTLE PLAN LIGHT PERFIL 0,4 M SO-AYP20L040...",20704
3,2 CONEXÕES PARA UNIVERSAL MEDIUM SO-AY103-0000,20721
4,4 GRAMPO FIXAÇÃO INTERMEDIÁRIA 39-44 E GRAMPO ...,20725


In [18]:
# Faça o merge entre a tabela "fradee" e a versão modificada da tabela "produtos"
Avarias = pd.merge(df_resultado, primeiros_produtos, on=' Produto ', how='left')
Avarias.head(10)

,Ticket,NF,Data Ocorrência,Ocorrência,Parceiro,Valor,Produto,Emissão_x,Quantidade,Categoria Produto,Emissão_y,UF Destino,SKU
0,#41080,24753,10/09/2024,Avaria,Viktoria,"R$ 482,67",Painel Solar Fotovoltaico Luxen Lnvu-555M (555Wp),02/09/2024,1,Painel 555 Wp,02/09/2024,SP,<NA>
1,#41024,24718,06/09/2024,Avaria,Solistica,"R$ 602,00",PAINEL SOLAR FOTOVOLTAICO LUXEN LNCT-585N TOPC...,30/08/2024,1,Painel 585 Wp,30/08/2024,SP,23047
2,#9103,36985,10/06/2020,Retorno,Alfa,"R$ 239,78",Painel Solar Fotovoltaico Resun RS6E-155M (155Wp),21/08/2024,1,Painel 155 Wp,NaN,<NA>,10141
3,#10358,46732,14/08/2020,Avaria,Viktoria,"R$ 863,18",PAINEL SOLAR FOTOVOLTAICO ZTROON ZTP340P (340WP),16/08/2024,2,Painel 340 Wp,10/08/2020,SP,22002
4,#10567,46841,25/08/2020,Avaria,Gran Cargo,"R$ 509,52",PAINEL SOLAR FOTOVOLTAICO RENESOLA (555WP) BR,15/08/2024,1,Painel 555 Wp,12/08/2020,RS,22098
5,#10755,45974,01/09/2020,Extravio,Tecmar,"R$ 863,18",Painel Solar Fotovoltaico BYD 340-PHK-36 Poli ...,09/08/2024,2,Painel 340 Wp,23/07/2020,PA,10001960
6,#36184,9196,08/01/2024,Extravio,Tecmar,"R$ 8.276,07",BATERIA DE Lítio UNIPOWER UPLFP48-100 (100AH) ...,01/08/2024,1,Bateria 100 Ah,07/12/2023,MG,20407
7,#11407,49871,25/09/2020,Extravio,Alfa,"R$ 287,48",Painel Solar Fotovoltaico Resun RSM020P (20Wp),29/07/2024,4,Painel 20 Wp,03/09/2020,TO,10143
8,#11533,50133,02/10/2020,Extravio,Alfa,"R$ 7.224,00",PAINEL SOLAR FOTOVOLTAICO LUXEN LNCT-585N TOPC...,24/07/2024,12,Painel 585 Wp,10/09/2020,PI,23047
9,#11565,50233,18/10/2020,Retorno,Alfa,"R$ 868,22",PAINEL SOLAR FOTOVOLTAICO SUNOVA SS-460-60MDH ...,17/07/2024,2,Painel 460 Wp,14/09/2020,MG,20768


In [19]:
Avarias.drop(columns='Emissão_y', inplace=True)
Avarias.rename(columns={'Emissão_x': ' Emissão ', 'UF Destino': 'UF'}, inplace=True)
Avarias.head(5)

,Ticket,NF,Data Ocorrência,Ocorrência,Parceiro,Valor,Produto,Emissão,Quantidade,Categoria Produto,UF,SKU
0,#41080,24753,10/09/2024,Avaria,Viktoria,"R$ 482,67",Painel Solar Fotovoltaico Luxen Lnvu-555M (555Wp),02/09/2024,1,Painel 555 Wp,SP,<NA>
1,#41024,24718,06/09/2024,Avaria,Solistica,"R$ 602,00",PAINEL SOLAR FOTOVOLTAICO LUXEN LNCT-585N TOPC...,30/08/2024,1,Painel 585 Wp,SP,23047
2,#9103,36985,10/06/2020,Retorno,Alfa,"R$ 239,78",Painel Solar Fotovoltaico Resun RS6E-155M (155Wp),21/08/2024,1,Painel 155 Wp,<NA>,10141
3,#10358,46732,14/08/2020,Avaria,Viktoria,"R$ 863,18",PAINEL SOLAR FOTOVOLTAICO ZTROON ZTP340P (340WP),16/08/2024,2,Painel 340 Wp,SP,22002
4,#10567,46841,25/08/2020,Avaria,Gran Cargo,"R$ 509,52",PAINEL SOLAR FOTOVOLTAICO RENESOLA (555WP) BR,15/08/2024,1,Painel 555 Wp,RS,22098


In [20]:

# Supondo que você já tenha seu dataframe carregado como Avarias
# Exemplo: Avarias = pd.read_csv('seu_arquivo.csv')

# Convertendo a coluna ' Emissão ' para datetime
Avarias[' Emissão '] = pd.to_datetime(Avarias[' Emissão '], format='%d/%m/%Y').dt.date

# Filtrando para dados posteriores a 01/01/2023
Avarias = Avarias[Avarias[' Emissão '] > pd.to_datetime('2023-01-01').date()]

# Exibindo as primeiras linhas do dataframe filtrado para confirmação
Avarias.head(5)


,Ticket,NF,Data Ocorrência,Ocorrência,Parceiro,Valor,Produto,Emissão,Quantidade,Categoria Produto,UF,SKU
0,#41080,24753,10/09/2024,Avaria,Viktoria,"R$ 482,67",Painel Solar Fotovoltaico Luxen Lnvu-555M (555Wp),2024-09-02,1,Painel 555 Wp,SP,<NA>
1,#41024,24718,06/09/2024,Avaria,Solistica,"R$ 602,00",PAINEL SOLAR FOTOVOLTAICO LUXEN LNCT-585N TOPC...,2024-08-30,1,Painel 585 Wp,SP,23047
2,#9103,36985,10/06/2020,Retorno,Alfa,"R$ 239,78",Painel Solar Fotovoltaico Resun RS6E-155M (155Wp),2024-08-21,1,Painel 155 Wp,<NA>,10141
3,#10358,46732,14/08/2020,Avaria,Viktoria,"R$ 863,18",PAINEL SOLAR FOTOVOLTAICO ZTROON ZTP340P (340WP),2024-08-16,2,Painel 340 Wp,SP,22002
4,#10567,46841,25/08/2020,Avaria,Gran Cargo,"R$ 509,52",PAINEL SOLAR FOTOVOLTAICO RENESOLA (555WP) BR,2024-08-15,1,Painel 555 Wp,RS,22098


In [21]:
Avarias.shape

(563, 12)

In [25]:
Avarias_faltantes = Avarias[Avarias["SKU"].isnull()]
Avarias_faltantes.head(15)

,Ticket,NF,Data Ocorrência,Ocorrência,Parceiro,Valor,Produto,Emissão,Quantidade,Categoria Produto,UF,SKU
0,#41080,24753,10/09/2024,Avaria,Viktoria,"R$ 482,67",Painel Solar Fotovoltaico Luxen Lnvu-555M (555Wp),2024-09-02,1,Painel 555 Wp,SP,<NA>
27,#12382,54133,17/12/2020,Extravio,Tecmar,"R$ 456,72",PAINEL SOLAR FOTOVOLTAICO CANADIAN HIKU CS3W (...,2024-06-20,1,Painel 450 Wp,RS,<NA>
74,#13586,54774,05/01/2021,Extravio,Movvi,"R$ 5,90",Kit Parafusos Martelo M8 - Rev.,2024-04-12,2,Estrutura,CE,<NA>
110,#15856,59748,29/04/2021,Extravio,Solistica,"R$ 4.658,42",,2024-03-22,1,None,BA,<NA>
209,#17374,67196,30/07/2021,Avaria,Movvi,"R$ 752,00",Cabo Fotovoltáico Lafeber 6mm2 Preto,2023-12-08,100,Cabo,SP,<NA>
214,#16872,65019,03/08/2021,Avaria,Tecmar,,,2023-12-08,,None,SP,<NA>
241,#18148,70053,13/09/2021,Avaria,Tecmar,,,2023-11-28,1,None,SP,<NA>
265,#18569,69604,06/10/2021,Extravio,Tecmar,"R$ 45,27",Cabo Fotovoltáico Lafeber 6mm2 Preto,2023-11-22,3,Cabo,PA,<NA>
280,#18875,72463,21/10/2021,Avaria,Tecmar,"R$ 83,88",Cabo Fotovoltáico Lafeber 6mm2 Preto,2023-11-17,12,Cabo,ES,<NA>
299,#19226,73366,09/11/2021,Avaria,Tecmar,"R$ 349,30",Fixador Final Regulável P. Master Rev,2023-11-14,2,Estrutura,PR,<NA>


In [23]:
Avarias.to_excel("N:/Drives compartilhados/28 - Dados BI/2 Dados Transformados/2.Frete/2.Frete_Avarias.xlsx")
Avarias_faltantes.to_excel("N:/Drives compartilhados/04 - OPERAÇÕES/70. Transporte/05_Casos de Avarias Extravios e Roubos/Avarias_SKU_faltantes.xlsx")

In [24]:
Avarias["SKU"].isnull().value_counts()

SKU
False    547
True      16
Name: count, dtype: int64